# Driving in Straight Line and maintain speed

In [1]:
print("Hello there")

Hello there


In [2]:
import carla
import time
import numpy as np
import math
import sys

In [3]:
import cv2

In [4]:
#connect to simulator
client = carla.Client('localhost', 2000)
world = client.get_world()

In [5]:
#utility script for destruction, clear all the actors (basically undo or destroy the vehicle)
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
#remove top and bottom qoutes to execute

In [6]:
'''
#get location of the client (I use this to confirm the connection with the sim)
spectator = world.get_spectator()
transform = spectator.get_transform()
spectator.set_transform(carla.Transform())
'''

'\n#get location of the client (I use this to confirm the connection with the sim)\nspectator = world.get_spectator()\ntransform = spectator.get_transform()\nspectator.set_transform(carla.Transform())\n'

In [7]:
'''
#print all the vehicles
for bp in world.get_blueprint_library().filter('vehicle'):
    print(bp.id)

vehicle.tesla.model3
'''

"\n#print all the vehicles\nfor bp in world.get_blueprint_library().filter('vehicle'):\n    print(bp.id)\n\nvehicle.tesla.model3\n"

In [8]:
# get the spawn points of world
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

In [9]:
#look for a blue print of your desired car, i want tesla
vehicle_bp = world.get_blueprint_library().filter('vehicle.tesla.model3')
#print(vehicle_bp)

#spawn the vehicle on start point
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [10]:
# position and create camera and open a live stream of car
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {
    'image': np.zeros((image_h, image_w, 4))
}

camera.listen(lambda image: camera_callback(image, camera_data))

In [11]:
# define constants 

#define speed constants
PREFERRED_SPEED = 30
SPEED_TRESHOLD = 2

#adding parameters to display text to the image
font = cv2.FONT_HERSHEY_SIMPLEX

#defining the lines and font to display
org = (30, 30)
org2 = (30, 50) # this line will be used for future steering angle
fontScale = 0.5
color = (255, 255, 255)
thickness = 1

In [12]:
# function to miantain the speed
def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_TRESHOLD:
        return 0.8
    else:
        return 0.4

In [13]:
# drive straight and maintain the speed

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

quit = False

while True:
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit - True
        break
    image = camera_data['image']
    steering_angle = 0
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    
    #display speed on window
    image = cv2.putText(image, 
                        'Speed: '+str(int(speed))+' kmh',
                        org2, font, fontScale, color, thickness, cv2.LINE_AA)

    # determine the input from current speed
    estimated_throttle = maintain_speed(speed)
    #now accelrate
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer = steering_angle))
    cv2.imshow('RGB Camera', image)

#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()